In [2]:
!pip install groq python-dotenv numpy tqdm datasets

In [3]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

In [4]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [5]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello there! I'm thrilled to meet you! I'm a math-loving assistant, here to help you with any math problems you might be struggling with. Whether it's algebra, geometry, calculus, or anything in between, I'm here to lend a hand.

So, what's on your mind? Do you have a specific math problem you're trying to solve, or perhaps a concept you're having trouble understanding? Let me know, and I'll do my best to assist you!


#### GSM8K 데이터셋 확인해보기

In [6]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [12]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [11]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama3-8b-8192",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer = extract_final_answer(response)

        if isinstance(predicted_answer, str):
            predicted_answer = predicted_answer.replace(",", "").strip()
            if predicted_answer:  # 빈 문자열 아닌 경우만 변환
                predicted_answer = float(predicted_answer)
            else:
                predicted_answer = 0  # 혹은 continue 또는 is_correct = False
            
            diff = abs(predicted_answer - correct_answer)
            is_correct = diff < 1e-5 if predicted_answer is not None else False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [9]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [ ]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [37]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(5)
VERBOSE = False

print(PROMPT)

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_5.txt")

Instruction:
Solve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.

[Example 1]
Question:
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Answer:72

[Example 2]
Question:
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?
Answer:10

[Example 3]
Question:
Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?
Answer:5

[Example 4]
Question:
Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read

 10%|█         | 5/50 [00:03<00:30,  1.48it/s]

Progress: [5/50]
Current Acc.: [40.00%]


 20%|██        | 10/50 [00:06<00:24,  1.66it/s]

Progress: [10/50]
Current Acc.: [30.00%]


 30%|███       | 15/50 [00:08<00:18,  1.86it/s]

Progress: [15/50]
Current Acc.: [20.00%]


 38%|███▊      | 19/50 [00:16<00:27,  1.14it/s]


KeyboardInterrupt: 

In [ ]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!

### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [14]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = "Instruction: Solve the following mathematical questions step by step. generate ONLY the answer after a tag, 'Answer:' with number. you should provide answer anyway which is not blank\n"

    for i in range(num_examples):
        idx = sampled_indices[i]
        cur_question = train_dataset['question'][idx]
        cur_answer = train_dataset['answer'][idx]

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"\n{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nLet's solve this step by step:\n"

    return prompt

In [ ]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_CoT_prompt(3)
VERBOSE = False

print(PROMPT)

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_3.txt")

Instruction: Solve the following mathematical questions step by step. generate ONLY the answer after a tag, 'Answer:' with number. you should provide answer anyway which is not blank

[Example 1]
Question:
Mary needs school supplies. She has 6 classes and needs 1 folder for each class. She also needs 3 pencils for each class. She decides that for every 6 pencils she should have 1 eraser. She also needs a set of paints for an art class.  Folders cost $6, pencils cost $2, and erasers cost $1. If she spends $80, how much did the set of paints cost in dollars?

Mary needs 6*1= <<6*1=6>>6 folders.
Mary needs 6*3= <<6*3=18>>18 pencils.
Mary needs 18/6= <<18/6=3>>3 erasers.
Mary spends 6*6= $<<6*6=36>>36 on folders.
Mary spends 18*2= $<<18*2=36>>36 on pencils.
Mary spends 3*1= $<<3*1=3>>3 on erasers.
Mary spends 36+36+3= $<<36+36+3=75>>75 on all the supplies except the paints.
Mary spends 80-75= $<<80-75=5>>5 on the paint set.
#### 5

[Example 2]
Question:
There are 235 books in a library. On

 10%|█         | 5/50 [00:12<02:58,  3.96s/it]

Progress: [5/50]
Current Acc.: [75.00%]


 20%|██        | 10/50 [00:59<05:26,  8.17s/it]

Progress: [10/50]
Current Acc.: [71.43%]


...

Progress: [40/50]
Current Acc.: [62.86%]


 90%|█████████ | 45/50 [06:16<00:45,  9.04s/it]

Progress: [45/50]
Current Acc.: [64.10%]


100%|██████████| 50/50 [06:54<00:00,  8.30s/it]

Progress: [50/50]
Current Acc.: [63.64%]


In [ ]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [7]:
def construct_my_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = "Instruction: You are a skillful mathmatician. Solve the following mathematical questions step by step. generate ONLY the answer after a tag, 'Answer:' with number.\n"

    for i in range(num_examples):
        idx = sampled_indices[i]
        cur_question = train_dataset['question'][idx]
        cur_answer = train_dataset['answer'][idx]

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"\n{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nLet's solve this step by step:\n"

    return prompt

In [ ]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_my_prompt(5)
VERBOSE = False

# print(PROMPT)

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "My_prompting_5.txt")

 10%|█         | 5/50 [00:03<00:34,  1.31it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [01:04<07:13, 10.85s/it]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [02:08<07:19, 12.56s/it]

...

Progress: [40/50]
Current Acc.: [67.50%]


 90%|█████████ | 45/50 [08:27<01:02, 12.57s/it]

Progress: [45/50]
Current Acc.: [68.89%]


100%|██████████| 50/50 [09:33<00:00, 11.46s/it]

Progress: [50/50]
Current Acc.: [72.00%]


In [ ]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!

### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!